In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### 
read each csv files and store them as a dataframe and take a look at first 5 rows of each one

In [2]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()  #it  has  9742 rows

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings_df.head() #it has 100836 rows

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# 
برای اینکه بتوانیم سال ساخت هر فیلم رو در یک ستون جداگانه در دیتافریم اول نگه داری کنیم، از روش زیر استفاده میکنیم که به ترتیب در خط اول از ستون تایتل، یک پرانتز باز و بسته که شامل4 دیجیت باشد رو استخراج میکنیم و داخل ستون جدیدی به نام سال ذخیره میکنیم ، در خط بعدی فقط 4 رقم اصلی سال رو از ستون سال استخراج میکنیم و داخل همون ستون و بجای سال با پرانتز جایگذاری میکنیم(یعنی در اصل پرانتزها رو حذف میکنیم)، در خط بعدی از ستون تایتل، پرانتز باز و بسته که شامل 4 رقم سال بود رو استخراج و پاک میکنیم و در خط آخر برای پاک کردن اسپیس های احتمالی در ستون تایتل از فانکشن استریپ استفاده کردیم 

In [4]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\3106782265.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


# 
تمام پایپ های ستون ژانر رو با تایع اسپلیت حذف میکنیم و دیتاهای هر سلول از این ستون رو تبدیل میکینم به یه آرایه کار کردن باهاش راحت تر بشه در ادامه 

In [5]:
movies_df['genres'] = movies_df.genres.str.split('|')

# 
از دیتافریم اول یک کپی میگیریم و با یک فور تو در تو (فور اول روی هر ایندکس از سطرهای دیتافریم اول حرکت میکنیم و با فور دوم توی هر سطر که هستیم، ستون ژانر رو در نظر میگیریم و برای اون ایندکس و اون ژانرهایی که داره رو توی دیتافریم جدیدمون و به ترتیب توی ستون مربوط به اسم اون ژانر عددش رو 1 میذاریم) و اینطوری ستون هایی از ژانرها برای هر فیلم اضافه میشه به دیتافریم جدید و توی هر سطر، ستون های ژانر مربوط به اون فیلم عدد 1 میگیره و بعد از اینکه روی تمام سطرها اینو اعمال کردیم و فور تموم شد، اون سلول های که خالی هستن رو با عدد 0 پر میکنیم که بتونیم تا حدودی به ماتریس مورد نظر(که ماتریسی هست که به ازای هر فیلم میاد مشخص میکنه 
چه ژانرهایی رو شامل میشه و چه ژانرهایی رو شامل نمیشه) شبیه کنیم دیتافریم مون رو

In [6]:
moviesWithGenres_df = movies_df.copy()

for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
        
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 
حالا برای دیتافریم امتیازها، ستون تایمزمپ رو حذف میکنیم چون نمیخوایم از این فیچر توی سیستم ریکامندرمون استفاده کنیم

In [7]:
ratings_df = ratings_df.drop('timestamp', 1)

C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\2083535515.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df = ratings_df.drop('timestamp', 1)


# 
در اینجا فرض میکنیم یه یوزر داریم که به یه سری فیلم ها یه سری امتیازهایی داده و میخواد که با سیستم ریکامندرمون یه فیلم جدید بهش پیشنهاد بدیم. بعد از گرفتن ورودی، اونو تو یه دیتافریم جدید به اسم اینپوت_موویز ذخیره میکنیم  

In [8]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


# 
روش کار به این صورته که ابتدا باید ببینیم هر فیلم چه ژانرهایی داره، و بر اساس امتیاز کاربر موردنظر به اون فیلم، ماتریس وزن دار برای هر فیلم رو بست بیاریم (با ساختن یه ماتریس جدید به اسم یوزرپروفایل که میاد میگه این کاربر در مجموع فیلم هایی که دیده به هر ژانر چه امتیازی داده(با ضرب کردن امتیاز کاربر به هر فیلم توی ستون ژانرهای مربوط به همون فیلم در نهایت محاسبه مجموع اعداد هر ستون برای پیدا کردن مجموع امتیازات کاربر به هر ژانر) و بعد مجموع امتیازاتش به هر ژانر رو نرمال سازی میکنیم که اعداد بین 0 و 1 بشه و بعد این ماتریس رو در ماتریس فیلم هایی که کاربرمون نگاه نکرده ضرب میکنیم که به ماتریس وزن دار به ازای هر فیلم برسیم و در نهایت این ماتریس رو به صورت  نزولی مرتب میکنیم که به ترتیب فیلم هایی که ستون وزن بالاتری دارن رو به کاربر پیشنهاد بدیم)  

# 

##  Adding genres & ID of each movie to inputMovies DataFrame 

# 
برای ساختن یوزر_پروفایل میایم از کل دیتافریم موویز_ویت_ژانر (که دیتافریمی بود که به ازای اسم هر فیلم، ژانرهایی که اون فیلم داره رو با عدد 1 توی ستون مربوط به اسم ژانرش مشخص میکرد) اونایی که تایتل شون(اسم فیلم) توی اطلاعات ورودی کاربر که تبدیل به لیست شده هست رو انتخاب میکنیم و تو یه دیتافریم جدید به اسم اینپوت_آیدی ذخیره میکینم. بعد از فانکشن مرج استفاده میکنیم برای اینکه بیاد تو دیتافریم اینپوت_موویز ستون هایی از دیتافریم اینپوت_آیدی که این دیتافریم نداره رو اضافه کنه بهش و بعد میایم از دیتافریم آپدیت شده، ستون های ژانر و سال و امتیاز رو حذف میکنیم چون لازمش نداریم. ولی ایراد احتمالی این کار اینه که باید دقیقا اسپلینگ اسم فیلم توی هر دوتا دیتافریم کاملا شبیه همدیگه باشه وگرنه نمیتونستیم از روی تطابق اسم هر فیلم به دیتافریم جدیدمون برسیم    

In [9]:
inputId = moviesWithGenres_df[moviesWithGenres_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)

inputMovies = inputMovies.drop('genres', 1).drop('year', 1)

inputMovies

C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\4127648860.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\4127648860.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('genres', 1).drop('year', 1)


,movieId,title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),rating
0,1,Toy Story,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
1,2,Jumanji,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,296,Pulp Fiction,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,1274,Akira,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
4,1968,"Breakfast Club, The",0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


# 

## Creating weighted Movie DataFrame

# 
اول ستون مووی_آیدی و تایتل و امتیاز رو حذف میکنیم که به دیتافریمی که فقط شامل  ژانرهایی از فیلم هایی که کابر دیده هست برسیم

In [10]:
userGenreTable = inputMovies.drop('movieId', 1).drop('title', 1).drop('rating', 1)
userGenreTable

C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\2088756995.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = inputMovies.drop('movieId', 1).drop('title', 1).drop('rating', 1)
C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\2088756995.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = inputMovies.drop('movieId', 1).drop('title', 1).drop('rating', 1)
C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\2088756995.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = inputMovies.drop('movieId', 1).drop('title', 1).drop('rating', 1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 
بعد میایم از دیتافریم اینپوت_موویز، امتیازی که کاربر به هر فیلم داده رو توی ماتریس یوزر_ژانر ضرب میکنیم که ماتریس یوزر پروفایل ساخته بشه. یعنی حالا به یه ماتریسی رسیدیم که بر اساس همه ژانرهای موجود، مشخص کرده که کاربر ما به هر ژانر در مجموع تمام فیلم هایی که دیده چه امتیازی داده

In [11]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

# 


# 
میایم یه جدول از کل آیدی های فیلم هایی که از اول اطلاعاتشون رو داشتیم میسازیم که دیگه به ازای هر آیدی تمام ژانرهاش رو نشون بده و برای اینکه بتونیم ضرب رو هم با همین جدول انجام بدیم، آیدی هر فیلم رو بعنوان ایندکس نگهداری میکنیم

In [12]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\Sp\AppData\Local\Temp\ipykernel_7620\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFr

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 
حالا میایم اون یوزر_پروفایلی که ساخته بودیم رو ضرب میکنیم توی جدول تمام فیلم ها همراه با ژانرشون که تو سلول قبلی کدش رو زدیم که به ازای هر مووی_آیدی که بعنوان ایندکس نگه داری میکردیم یه عدد بین 0 و 1 میده که میزان علاقه کاربر به هر فیلم رو مشخص میکنه دیگه(طبق توضیحات مفصلی که تو قسمت های قبلی داده بودم)

In [14]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

# 
حالا باید این جدول رو به صورت نزولی مرتب کنیم(طبق توضیحاتی که قبلا داده بودم)

In [15]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
134853    0.734266
148775    0.685315
117646    0.678322
6902      0.678322
81132     0.671329
dtype: float64

# 
بعد میایم بر اساس ایندکس های جدول بالا(که آیدی هر فیلم بود) مشخص میکنیم هر آیدی از جدول مرتب شده ی بالا، مربوط  به کدوم فیلم هست و به ترتیب چه فیلم هایی رو کاربرمون بیشتر از همه دوست داره و بهش پیشنهاد میدیم، اینطوری که میایم لوکیشن 20 تا مووی_آیدی اول از دیتافریم ریکامندیشن_تیبل رو از دیتافریم موویز پیدا میکنیم و با اطلاعات کاملش به کاربر پیشنهاد میدیم 

In [16]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1390,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4631,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
5819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
6047,40339,Chicken Little,"[Action, Adventure, Animation, Children, Comed...",2005
6448,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed...",2007
6455,52287,Meet the Robinsons,"[Action, Adventure, Animation, Children, Comed...",2007
